# Restricted Boltzmann Machines

The explanations in this notebook are taken from: 
- [Deep Learning meets Physics: Restricted Boltzmann Machines Part I (Towards Data Science)](https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-i-6df5c4918c15)

- [Deep Learning meets Physics: Restricted Boltzmann Machines Part II (Towards Data Science)](https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-ii-4b159dce1ffb)

- [Restricted Boltzman machine (Wikipedia)](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine)

## Architecture of a RBM
<center><img src="./images/rbm_architecture.png" width=700 class="center"></center>

One purpose of deep learning models is to encode dependencies between variables. The capturing of dependencies happen through associating of a scalar energy to each configuration of the variables, which serves as a measure of compatibility. A high energy means a bad compatibility. An energy based model model tries always to minimize a predefined energy function. The energy function for the RBMs is defined as (**Eq. 1**):

<center><img src="./images/rbm_eq1.png" width=400 class="center"></center>

At each point in time the RBM is in a certain state. The state refers to the values of neurons in the visible and hidden layers $\mathbf{v}$ and $\mathbf{h}$. The probability that a certain state of $\mathbf{v}$ and $\mathbf{h}$ can be observed is given by the following joint distribution (**Eq. 2**):

<center><img src="./images/rbm_eq2.png" width=200 class="center"></center>

Here $\mathbf{Z}$ is called the ‘partition function’ that is the summation over all possible pairs of visible and hidden vectors.

Unfortunately it is very difficult to calculate the joint probability due to the huge number of possible combination of $\mathbf{v}$ and $\mathbf{h}$ in the partition function $\mathbf{Z}$. Much easier is the calculation of the conditional probabilities of state $\mathbf{h}$ given the state $\mathbf{v}$ and conditional probabilities of state $\mathbf{v}$ given the state $\mathbf{h}$ (**Eq. 3**):

<center><img src="./images/rbm_eq3.png" width=200 class="center"></center>

It should be noticed that each neuron in a RBM can only exist in a binary state of 0 or 1. Given an input vector $\mathbf{v}$ the probability for a single hidden neuron $j$ being activated is (**Eq. 4**):

<center><img src="./images/rbm_eq4.png" width=450 class="center"></center>

Here $\sigma$ is the Sigmoid function. This equation is derived by applying the Bayes Rule to Eq. 3.

Analogous the probability that a binary state of a visible neuron $i$ is set to 1 is (**Eq. 5**):

<center><img src="./images/rbm_eq5.png" width=450 class="center"></center>

## Training of a RBM
Restricted Boltzmann machines are trained to maximize the product of probabilities assigned to some training set $\mathbf{V}$ (a matrix, each row of which is treated as a visible vector $\mathbf{v}$),

<center><img src="./images/rbm_argmaxP.svg" width=150 class="center"></center>

or equivalently, to maximize the expected log probability of a training sample $\mathbf{v}$ selected randomly from $\mathbf{V}$:

<center><img src="./images/rbm_argmaxEP.svg" width=200 class="center"></center>

The algorithm most often used to train RBMs, that is, to optimize the weight vector $\mathbf{W}$ is the contrastive divergence (CD). The algorithm performs Gibbs sampling and is used inside a gradient descent procedure to compute weight update.

The basic, single-step contrastive divergence (CD-1) procedure for a single sample can be summarized as follows:

1. Take a training sample $v$, compute the probabilities of the hidden units and sample a hidden activation vector $h$ from this probability distribution.
2. Compute the outer product of $v$ and $h$ and call this the *positive gradient*.
3. From $h$, sample a reconstruction $v'$ of the visible units, then resample the hidden activations $h'$ from this. (Gibbs sampling step)
4. Compute the outer product of $v'$ and $h'$ and call this the *negative gradient*.
5. Let the update to the weight matrix $\mathbf{W}$ be the positive gradient minus the negative gradient, times some learning rate: 
<center><img src="./images/rbm_deltaW.svg" width=200 class="center"></center>
6. Update the biases $a$ and $b$ analogously:
<center><img src="./images/rbm_deltaA.svg" width=150 class="center"></center>

To sum up, the update to the weight matrix is:

$$
    \Delta w_{ij} = \epsilon\left( <v_ih_j>_{data} - <v_ih_j>_{reconstruction} \right)
$$



In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import wget
import struct

In [ ]:
wget.download("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", out="./data/MNIST/X_train.gz")
wget.download("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", out="./data/MNIST/Y_train.gz")
wget.download("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", out="./data/MNIST/X_test.gz")
wget.download("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", out="./data/MNIST/Y_test.gz")

In [ ]:
%%bash
gzip -d ./data/MNIST/X_train.gz
gzip -d ./data/MNIST/Y_train.gz
gzip -d ./data/MNIST/X_test.gz
gzip -d ./data/MNIST/Y_test.gz

In [2]:
# reading IDX files. Taken from https://gist.github.com/tylerneylon/ce60e8a06e7506ac45788443f7269e40
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

X_train = read_idx("./data/MNIST/X_train")
X_train = X_train.reshape((X_train.shape[0], -1))
Y_train = read_idx("./data/MNIST/Y_train")
X_test = read_idx("./data/MNIST/X_test")
X_test = X_test.reshape((X_test.shape[0], -1))
Y_test = read_idx("./data/MNIST/Y_test")

In [3]:
num_hidden = 10
num_visible = X_train.shape[1]

max_epochs = 10000
learning_rate = 0.1

In [ ]:
# initialize a weight matrix of dimensions (num_visible x num_hidden)
np_rng = np.random.RandomState(42)

weights = np_rng.uniform(low=-0.1 * np.sqrt(6. / (num_hidden + num_visible)), 
                         high=0.1 * np.sqrt(6. / (num_hidden + num_visible)), 
                         size=(num_visible, num_hidden))

# adding bias weights
hidden_bias = np.zeros(num_hidden)
visible_bias = np.zeros(num_visible)

In [5]:
class RBM:
    def __init__(self, n_visible, n_hidden):
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.W = np.random.uniform(
            low=-4 * np.sqrt(6. / (n_hidden + n_visible)),
            high=4 * np.sqrt(6. / (n_hidden + n_visible)),
            size=(n_visible, n_hidden))
        
        self.hbias = np.zeros(n_hidden)
        self.vbias = np.zeros(n_visible)
    
    def energy(self, v_sample):
        # visible term
        vis_term = np.dot(v_sample, self.vbias)
        hid_term = np.dot(self.)

array([[ 8.37301852e-01, -1.61189461e-01,  8.09058470e-01,
         3.15347032e-01,  5.44081967e-01,  1.51255054e-01,
         6.77139995e-02, -8.93841479e-01,  3.64720785e-01,
         9.20312655e-01],
       [ 7.21363040e-01,  9.25835172e-01,  3.50069082e-01,
         7.55475231e-01,  5.68178144e-01, -6.34287048e-01,
         6.35317939e-02,  6.50532829e-01,  6.33331683e-01,
        -3.66808111e-01],
       [-5.22621209e-01, -1.39182735e-01, -4.50999706e-01,
        -5.31874888e-01,  3.79087525e-01, -2.70689648e-01,
        -3.79175102e-01,  6.21218939e-01,  7.48270620e-01,
         8.27895678e-01],
       [ 3.93133708e-01,  6.16684135e-01,  3.24625737e-01,
        -9.12561892e-01, -5.69034645e-01,  7.53194783e-01,
         7.18500327e-01, -2.73246939e-01,  3.58552536e-01,
        -5.16377276e-01],
       [-1.72452735e-02,  7.60798852e-01,  6.03648178e-01,
         5.93768439e-01, -2.25240784e-01, -7.52728762e-01,
         8.91067853e-01,  6.86768299e-01,  4.34913122e-01,
         5.

In [ ]:
def logistic(x):
    return 1.0 / (1 + np.exp(-x))

errors = []

for epoch in tqdm(range(max_epochs)):    
    # X: (num_samples x num_features)
    # W: (num_features x num_hidden)
    
    # Positive gradient phase
    pos_hidden_activations = np.dot(X_train, weights) + hidden_bias                         # H = XW + hb   (num_samples x num_hidden)
    pos_hidden_probs = logistic(pos_hidden_activations)                                     # H = logistic(XW + hb) (num_samples x num_hidden)
    pos_hidden_states = pos_hidden_probs > np.random.rand(data.shape[0], num_hidden + 1)    # HS = binarize(H)
    
    # Note that we're using the activation *probabilities* of the hidden states, not the hidden states       
    # themselves, when computing associations. We could also use the states; see section 3 of Hinton's 
    # "A Practical Guide to Training Restricted Boltzmann Machines" for more.
    pos_associations = np.dot(data.T, pos_hidden_probs)                                     # A = (X^T)H   (n_features x num_hidden)
    
    # Negative gradient phase
    neg_visible_activations = np.dot(pos_hidden_states, weights.T)                          # R = H(W^T) (num_samples x num_features)
    neg_visible_probs = logistic(neg_visible_activations)                                   # R = logistic(H(W^T)) (num_samples x num_features)
    neg_visible_probs[:,0] = 1                                                              # Fix the bias unit. (num_samples x num_features)
    neg_hidden_activations = np.dot(neg_visible_probs, weights)                             # H' = RW (num_samples x num_hidden)
    neg_hidden_probs = logistic(neg_hidden_activations)                                     # H' = logistic(RW) (num_samples x num_hidden)
    # Note, again, that we're using the activation *probabilities* when computing associations, not the states themselves.
    neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)                        # A' = (R^T)H'  (n_features x num_hidden)
    
    
    # Update weights
    weights = learning_rate * ((pos_associations - neg_associations) / data.shape[0])       # W' = lr*(A - A') / num_samples
    
    # error computation
    error = np.sum((data - neg_visible_probs)**2)
    errors.append(error)
    print(error)
    
plt.figure(figsize=(25, 5))
plt.plot(errors)
plt.show()